## Copyright 2023 Google LLC

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# StyleAligned over Hunyuan

In [ ]:
import sys
import os
sys.path.append('/home/supermicro/zyb/Unified-codebase-for-proccessing-the-single-image/style-aligned-for-DiT')

In [ ]:
from diffusers import HunyuanDiTPipeline, DDIMScheduler
import torch
import mediapy
import sa_handler_for_hunyuan

In [ ]:
# init models

scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False,
                              set_alpha_to_one=False)
pipeline = HunyuanDiTPipeline.from_pretrained("Tencent-Hunyuan/HunyuanDiT-v1.2-Diffusers", torch_dtype=torch.float16).to("cuda:1")

In [ ]:
handler = sa_handler_for_hunyuan.Handler(pipeline)
sa_args = sa_handler_for_hunyuan.StyleAlignedArgs(share_group_norm=False,
                                      share_layer_norm=False,
                                      share_attention=False,
                                      adain_queries=False, # origing True
                                      adain_keys=True, # origin True
                                      adain_values=False,
                                      shared_score_scale=1,
                                     )

handler.register(sa_args, )

In [ ]:
# run StyleAligned

# sets_of_prompts = [
#   "a toy train. macro photo. 3d game asset",
#   "a toy airplane. macro photo. 3d game asset",
#   "a toy bicycle. macro photo. 3d game asset",
#   "a toy car. macro photo. 3d game asset",
#   "a toy boat. macro photo. 3d game asset",
# ]

# sets_of_prompts = [
#   "a cat in minimal flat design illustartion",
#   # "a dog in minimal flat design illustartion",
#   # "a bird in minimal flat design illustartion",
#   # "a fish in minimal flat design illustartion",
#   # "a unicorn in minimal flat design illustartion",
# ]

sets_of_prompts = [
  "A brown and white corgi sits on an orange background, looking up at the camera. The photo is close-up, centered, and head-up, showing the real corgi and creating a warm atmosphere.",
  "A brown and white corgi is swimming. The photo is close-up, centered, and head-up, showing the real corgi and creating a warm atmosphere.",
  "A brown and white corgi sits on an orange background, wearing a hat, looking up at the camera. The photo is close-up, centered, and head-up, showing the real corgi and creating a warm atmosphere.",
  "A brown and white corgi in a bucket, looking up at the camera. The photo is close-up, centered, and head-up, showing the real corgi and creating a warm atmosphere.",
  "A brown and white corgi sits, in the style of van gogh, looking up at the camera. The photo is close-up, centered, and head-up, creating a warm atmosphere.",
]


images = pipeline(sets_of_prompts,).images
mediapy.show_images(images)

In [ ]:
for i in range(len(images)):
    image = images[i]
    image.save('/home/supermicro/zyb/Unified-codebase-for-proccessing-the-single-image/style-aligned-for-DiT/results/only_adain_keys_corgi_{}.png'.format(sets_of_prompts[i].replace(' ', '_')))

In [ ]:
print(pipeline)
transformer = pipeline.transformer
print(transformer.attn_processors.keys())
num_self_layers = len([name for name in transformer.attn_processors.keys() if 'attn1' in name])
print(num_self_layers)